In [1]:
from __future__ import annotations
import sys; sys.path.insert(0, '..')

%load_ext autoreload
%autoreload 2


# python
import os
import ssl
import csv

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import session_info

from pathlib import Path
from inspect import cleandoc

# utils
from utils import Constants
from modules.preprocesing import preprocess

# text
import re
import spacy
import unidecode
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

from sklearn.decomposition import LatentDirichletAllocation

# sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# stat
from scipy import stats

# statsmodels
from statsmodels.stats.multicomp import pairwise_tukeyhsd


# typings
from pandas import DataFrame as PandasDF
from typing import List, Dict, Union

# setup
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('max_colwidth', None)
# decimals
np.set_printoptions(precision=6)

In [4]:
# cargar el dataset sklearn
if not os.environ.get('CI'):
    ssl._create_default_https_context =\
        ssl._create_unverified_context
          
# rutas absolutas
here: Path = Path.cwd().absolute().parent
data: Path = here / 'data'
poetry_fundation_cleaned: Path = data / 'CleanedPoetryFoundationData.csv'
cv_poetry: Path = data / 'vallejo_poems_en.csv'

In [5]:
setup_load:Dict = dict(
    sep=Constants.PIPE_STR,
    quotechar='"',
    quoting=csv.QUOTE_NONNUMERIC,
    encoding=Constants.ENCODING
)

if not poetry_fundation_cleaned.is_file() or not cv_poetry.is_file():
    raise FileNotFoundError(
        cleandoc(f'''
        El archivo {poetry_fundation_cleaned} no existe.
        Por favor, descargue el archivo desde:
        https://www.kaggle.com/datasets/abhinavwalia95/poetryfoundationorg
        y coloquelo en la carpeta data.
        ''')
    )
    
poetry_df: PandasDF = (
    pd.read_csv(
        str(poetry_fundation_cleaned), 
        **setup_load
    )
)

cv_df: PandasDF = (
        pd.read_csv(
        str(cv_poetry), 
        **setup_load
    )
)

cv_df[['title', 'poem']] = (
        cv_df[['title', 'poem']]
        .apply(lambda col: col.astype(str).apply(preprocess))
    )

poetry_df = poetry_df.loc[~poetry_df.poem.isna(),:]

In [6]:
display(poetry_df.head(3))

,title,poem,poet,tags
0,invis fish,invis fish swim ghost ocean describ wave sand waterworn rock soon fish learn walk human come ashor paint dream die stone later later ocean floor punctuat chevi truck carri dreamer decend go store,joy harjo,"['living', 'time brevity', 'relationships', 'family ancestors', 'nature', 'landscapes pastorals', 'seas', 'rivers', 'streams', 'social commentaries', 'history politics']"
1,not bother earth spirit,not bother earth spirit live work stori oldest stori world delic chang see watch invit coffe warm bread oblig stay listen ordinari stori endur earthquak lightn death love blind beauti stori compel want leav trap stone finger escap,joy harjo,"['religion', 'the spiritual', 'mythology folklore', 'fairytales legends']"
2,hour consid hydrangea,hour consid hydrangea salt sand plant variet question variet diet mother know pound feel like lose lose ye sens possibl beauti grow extern extern beauti beauti occur surfac plant sun darken skin child small beauti obviou beauti hand swell bite spread insect venom small appear feel smash skull floor scream hold lap kitchen floor open freezer press pack frozen clay forehead like cold obviou hydrangea walk push child stroller walk push haul lift have bodi adjunct bodi compos errand weight tender small power imagin feel small weight pound like interf twitch muscl bodi object mother confus middleag mother littl spare flesh feel inch major muscl pull graviti weight child sleep hour think hydrangea let man look stop brush drowsi child littl eye face bare consid mother consid miss subtl power differenti time mass apprehen,simon white,"['living', 'parenthood', 'the body', 'the mind', 'nature', 'trees flowers']"


In [7]:
display(cv_df.head(3))

,title,poem
0,black herald,blow life power not know blow god hatr backlash suffer dam soul not know open dark furrow fierce face strong mayb hors barbar attila black herald death send deep abyss soul christ rever faith destini blasphem gori blow crackl bread burnsup oven door man poor poor turn eye slap shoulder call turn craze eye live dam like pond guilt gaze blow life power not know
1,black stone white stone,shall die pari rainstorm day rememb shall die pari bother doubtless thursday like today autumn shall thursday today thursday line set shoulder evil like today turn head journey way cesar vallejo dead strike hit hard stick hard end rope wit thursday shoulder bone loneli rain road
2,pari octob poem,leav bench away pant great situat action number split leav champ elyse strang alley moon make turn death goe away cradl leav surround peopl cut loo human resembl turn dispatch shadow away remain creat alibi shoe eyelet mud bend elbow button shirt
